<a href="https://colab.research.google.com/github/ScotsDunn/Data-Science-2002/blob/main/Project%201/DS_2002_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DS 2002 Project 1
Ashley Nguyne (rhw9tv) and Scotty Dunn (jgr6gg)

CSV = https://www.kaggle.com/datasets/singhnavjot2062001/11000-medicine-details

JSON = https://www.kaggle.com/datasets/bricevergnou/spotify-recommendation

## Import data and convert to dataframe


In [ ]:
#Import necessary packages
import pandas as pd
from google.colab import files
from urllib.request import urlopen
import json
import sqlite3

#Fetch data from github and convert to dataframes
csvURL = 'https://github.com/ScotsDunn/DS2002-Data-Project1/raw/refs/heads/main/Data/Medicine_Details.csv'
try:
  medicine_df = pd.read_csv(csvURL,index_col=0)
except:
  print("Error loading in medicine csv: make sure that github raw URL is active and valid")

jsonURL = "  https://github.com/ScotsDunn/DS2002-Data-Project1/raw/refs/heads/main/Data/Spotify%20Data.json"
try:
  response = urlopen(jsonURL)
  spotify_json = json.loads(response.read())
  spotify_df = pd.DataFrame(spotify_json['audio_features'])
except:
  print("Error loading in spotify csv: make sure that github raw URL is active and valid")

##Delete columns from dataframe based on what user wants

In [ ]:
#Print number of rows and columns in dataframes before modifications
print("Medicine dimensions pre-modification")
m_rows = len(medicine_df.index)
m_cols = len(medicine_df.columns)
print("Medicine rows: ", m_rows)
print("Medicine columns: ", m_cols)
print("Medicine column names:" + str(medicine_df.columns.tolist()))

print("------")

print("Spotify dimensions pre-modification")
s_rows = len(spotify_df.index)
s_cols = len(spotify_df.columns)
print("Spotify rows: ", s_rows)
print("Spotify columns: ", s_cols)
print("Spotify column names:" + str(spotify_df.columns.tolist()))

Medicine dimensions pre-modification
Medicine rows:  11825
Medicine columns:  8
Medicine column names:['Composition', 'Uses', 'Side_effects', 'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %']
------
Spotify dimensions pre-modification
Spotify rows:  100
Spotify columns:  18
Spotify column names:['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']


In [ ]:
#Function that can delete column from any dataframe based on user input
def deleteCol(df, df_name):
  answer = ""
  while(answer != "Y" and answer != "N"): #Runs while loop until user enters Y or N
    print(f"\nWould you like to delete a column from {df_name} Y/N")
    answer = input()
    if(answer == "Y"): #If user answers Y to wanting to delete a column, enters IF statement to take input and delete that column
      print(f"{df_name} Column names:" + str(df.columns.tolist()))
      print(f"\nWhat column do you want to delete?")
      answer = str(input())
      try:    #Tests to see if column exists
        del df[answer]
      except:
        print("Column does not exist")

    print(f"\n{df_name} dimensions post modification")
    new_mrows = len(df.index)
    new_mcols = len(df.columns)
    print(f"New {df_name} rows: ", new_mrows)
    print(f"New {df_name} columns: ", new_mcols)

#Run delete column functions on both dataframes
deleteCol(medicine_df, "Medicine Dataframe")
print("--------")
deleteCol(spotify_df, "Spotify Dataframe")


Would you like to delete a column from Medicine Dataframe Y/N
Y
Medicine Dataframe Column names:['Composition', 'Uses', 'Side_effects', 'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %']

What column do you want to delete?
Image URL

Medicine Dataframe dimensions post modification
New Medicine Dataframe rows:  11825
New Medicine Dataframe columns:  7

Would you like to delete a column from Medicine Dataframe Y/N
Y
Medicine Dataframe Column names:['Composition', 'Uses', 'Side_effects', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %']

What column do you want to delete?
N
Column does not exist

Medicine Dataframe dimensions post modification
New Medicine Dataframe rows:  11825
New Medicine Dataframe columns:  7
--------

Would you like to delete a column from Spotify Dataframe Y/N
Y
Spotify Dataframe Column names:['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'livenes

## Convert dataframe to datatype of choice

In [ ]:
#function that can convert any dataframe into a csv, json, or db file based on user input
def convert(df, filename):
  answer = ""
  while(answer != "Y" and answer != "N"):
    print(f"Would you like to convert {filename} to a CSV file? (Y/N)") #Ask user if they want to convert df to csv
    answer = str(input()).upper()
  if(answer == "Y"):
    file = str(input("What would you like the filename to be?\n")) + ".csv"   #gets filename from user and adds correct filetype
    try:
      df.to_csv(file) #converts dataframe to csv
      files.download(file) #google colab function to download file
    except:
      print(f"Error downloading {file}")

  answer = ""
  while(answer != "Y" and answer != "N"):
    print(f"\nWould you like to convert {filename} to a JSON file? (Y/N)") #Ask user if they want to convert df to json
    answer = str(input()).upper()
  if(answer == "Y"):
    df.reset_index(drop=True, inplace=True) #Was having issues where df with index could not convert to index. Fixed by simply removing indexes
    file = str(input("What would you like the filename to be?\n")) + ".json" #gets filename from user and adds correct filetype
    try:
      df.to_json(file) #converts dataframe to json
      files.download(file) #google colab function to download file
    except:
      print(f"Error downloading {file}")

  answer = ""
  while(answer != "Y" and answer != "N"):
    print(f"\nWould you like to convert {filename} to a SQL database? (Y/N)")
    answer = str(input()).upper()
  if(answer == "Y"):
    file = str(input("\nWhat would you like the filename to be?\n")) + ".db" #gets filename from user and adds correct filetype
    table = str(input("What would you like the table name to be?\n")) #database structures also require a table name to initialize
    try:
      conn = sqlite3.connect(file) #creates connection
    except:
      print("Could not form connection to sqlite database")

    try:
      df.to_sql(table, conn, if_exists='replace', index=False) #converts dataframe to database
      conn.close() #close connection
      files.download(file) #downloads file
    except:
      print(f"Error downloading {file}")

  print("------")

## Run program

In [ ]:
#Calls convert functions for both dataframes
convert(medicine_df, "medicine_df")
convert(spotify_df, "spotify_df")

Would you like to convert medicine_df to a CSV file? (Y/N)
Y
What would you like the filename to be?
spotify_csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Would you like to convert medicine_df to a JSON file? (Y/N)
Y
What would you like the filename to be?
spotify_json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Would you like to convert medicine_df to a SQL database? (Y/N)
spotify_sql

Would you like to convert medicine_df to a SQL database? (Y/N)
Y

What would you like the filename to be?
spotify_sql
What would you like the table name to be?
Spotify


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------
Would you like to convert spotify_df to a CSV file? (Y/N)
N

Would you like to convert spotify_df to a JSON file? (Y/N)
N

Would you like to convert spotify_df to a SQL database? (Y/N)
N
------
